In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'retina'

In [ ]:
RANDOM_SEED = 666

In [ ]:
pd.set_option("display.max_columns", None)

# Airline Passenger Satisfaction

From last week, we can look at the dataset corresponding to predicting whether passengers were satisfied with the airline. We read in both a training and test set.

In [ ]:
train = pd.read_csv("../../04/data/airline_satisfaction/train.csv", index_col=0)
train.head()

In [ ]:
test = pd.read_csv("../../04/data/airline_satisfaction/test.csv", index_col=0)
test.head()

## Linear Model

To start, we train a logistic regression model using only numerical features.

In [ ]:
# Train a simple model using only numerical features
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
ignore_features = ["id"]
features = [
    column
    for column, series in train.items()
    if np.issubdtype(series.dtype, np.number) and column not in ignore_features
]
target = "satisfaction"

In [ ]:
X_train = train[features]

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train[target])

In [ ]:
X_test = test[features]
y_test = label_encoder.transform(test[target])

In [ ]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("imputer", SimpleImputer()),
        ("estimator", LogisticRegression(random_state=RANDOM_SEED)),
    ]
)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_train_pred = pipeline.predict(X_train)

In [ ]:
y_test_pred = pipeline.predict(X_test)

Let's look at the classification report

In [ ]:
from sklearn.metrics import classification_report

print("Training")
print(classification_report(y_train, y_train_pred))
print("Test")
print(classification_report(y_test, y_test_pred))

Model has decent performance, no real signs of overfitting.

### Random Forest

Now, let's try out a random forest on the data using the default hyperparameters.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer()),
        # NOTE: We do not have to scale our data
        ("estimator", RandomForestClassifier(random_state=RANDOM_SEED)),
    ]
)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [ ]:
print("Training")
print(classification_report(y_train, y_train_pred))
print("Test")
print(classification_report(y_test, y_test_pred))

Wow! It's definitely overfit to the training data, but it performs quite well on the test data. Let's see what's going on inside

In [ ]:
model = pipeline.named_steps["estimator"]
model

In [ ]:
print("Model (hyper)parameters")
print(f"{'n_estimators':25s} = {model.n_estimators}")
for param in model.estimator_params:
    print(f"{param:25s} = {getattr(model, param)}")

The random forest is made up of 100 individual decision trees. Here's the first one:

In [ ]:
model.estimators_[0]

And here are some details about that tree

In [ ]:
tree = model.estimators_[0]

In [ ]:
for param in ("max_depth", "n_leaves", "node_count"):
    print(f"{param:25s} = {getattr(tree.tree_, param):,}")

Lastly, let's see what it looks like:

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

dot_data = export_graphviz(
    tree,
    out_file=None,
    feature_names=features,
    impurity=False,
    class_names=label_encoder.classes_,
)
graph = graphviz.Source(dot_data)
graph

### Grid Search

Was this the best model we could have produced? Let's try running a cross validation grid search over some key hyperparameters. This might take a little bit to run!

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "estimator__n_estimators": [5, 10, 100, 200, 500],
    "estimator__max_depth": [2, 10, 25, None],
}
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer()),
        ("estimator", RandomForestClassifier(random_state=RANDOM_SEED)),
    ]
)
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    return_train_score=True,
    refit=True,
    verbose=1
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
results["param_estimator__max_depth"] = results["param_estimator__max_depth"].apply(
    lambda x: f"{x:02d}" if x is not None else "none"
)

Let's investigate the results by looking at the average _training_ F1 score across all cross validation folds as a function of the two hyperparameters.

In [ ]:
print("Grid Search Results for Training F1 Score")
(
    pd.pivot_table(
        results,
        index="param_estimator__n_estimators",
        columns="param_estimator__max_depth",
        values="mean_train_score",
    ).style.background_gradient(cmap="summer", axis=None)
)

It looks like the `max_depth` is more important than `n_esimtators`, but it's still important that both have high values for good performance on the training data. 

Now, let's look at the average _validation_ F1 score.

In [ ]:
print("Grid Search Results for Validation F1 Score")

(
    pd.pivot_table(
        results,
        index="param_estimator__n_estimators",
        columns="param_estimator__max_depth",
        values="mean_test_score",
    ).style.background_gradient(cmap="summer", axis=None)
)

It seems that we likely aren't fully overfitting -- both training and test F1 scores continued to increase with increasing hyperparameter values.

If we again run the classification report on the full training and the final test set, we actually see that model performance hasn't changed from the original model. I guess we had a pretty optimal model all along!

In [ ]:
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

In [ ]:
print("Training")
print(classification_report(y_train, y_train_pred))
print("Test")
print(classification_report(y_test, y_test_pred))

Lastly, we can inspect the best models' feature importances:

In [ ]:
fig, ax = plt.subplots()
feature_importances = grid_search.best_estimator_.named_steps["estimator"].feature_importances_
sort_idx = np.argsort(feature_importances)
sorted_features = [features[idx] for idx in sort_idx]
ax.barh(sorted_features, feature_importances[sort_idx])
ax.set_title("Feature Importances")
None